В задании вам понадобится собрать генеративную модлель для языка

---

# LSTM (7 баллов)

В данной части нужно реализовать модель с ипользованием LSTM

In [1]:
%pip install --quiet sentencepiece datasets transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch.utils.data import DataLoader
from dataset_maker import DatasetMaker, collate_batch
from data_generator import DataGenerator

/home/super-pc2/Документы/test/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ниже вам нужно реализовать модель, которая по началу последовательности предсказывает следующий токен.
*   Модель получает на вход последовательность токенов, прогоняет её через LSTM и выдает вероятности следующего токена.  
*   Используйте LSTM из pytorch
*   Не забудьте про `batch_first`

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
DIRECTORY = './data/books_txt'  
VOCAB_PATH = './data/vocab.txt'
MAX_SEQ_LEN = 256
BATCH_SIZE = 32
OFFSET = MAX_SEQ_LEN // 2
TEST_SIZE = 0.03

In [5]:
dataset_maker = DatasetMaker(directory=DIRECTORY, 
                             test_size=TEST_SIZE, 
                             vocab_path=VOCAB_PATH, 
                             max_seq_len=MAX_SEQ_LEN, 
                             batch_size=BATCH_SIZE, 
                             offset=OFFSET, 
                             create_test_segments=True)
train_dataloader = DataLoader(dataset_maker, 
                              batch_size=BATCH_SIZE, 
                              shuffle=True, 
                              collate_fn=collate_batch)

In [6]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, segments_test):
        self.segments_test = segments_test

    def __len__(self):
        return len(self.segments_test)

    def __getitem__(self, idx):
        return self.segments_test[idx]

test_dataset = TestDataset(dataset_maker.segments_test)

test_dataloader = DataLoader(
    dataset=test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=collate_batch
)

In [8]:
def check_overlaps(train_segments, test_segments):
    train_set = set(tuple(seg) for seg in train_segments)
    test_set = set(tuple(seg) for seg in test_segments)
    overlaps = train_set.intersection(test_set)
    return len(overlaps) > 0, overlaps

overlap_exists, overlapping_segments = check_overlaps(dataset_maker.segments, dataset_maker.segments_test)
if overlap_exists:
    print(f"Overlaps detected between training and test segments: {overlapping_segments}")
else:
    print("No overlaps detected between training and test segments.")

print(f"Number of training texts: {len(dataset_maker.texts)}")
print(f"Number of test texts: {len(dataset_maker.texts_test)}")
print(f"Number of training segments: {len(dataset_maker.segments)}")
print(f"Number of test segments: {len(dataset_maker.segments_test)}")

No overlaps detected between training and test segments.
Number of training texts: 726
Number of test texts: 23
Number of training segments: 273510
Number of test segments: 3549


In [24]:
LEARNING_RATE = 0.0001
EMBEDDING_DIM = 256
NUM_LAYERS = 2
PAD_TOKEN_IDX = 0
SAVE_PATH = './saved_models'
BEST_MODEL_PATH = './saved_models/best_model.pth'

In [10]:
model = DataGenerator(vocab_size=dataset_maker.tokenizer.vocab_size, test_dataloader=test_dataloader, embed_dim=EMBEDDING_DIM, num_layers=NUM_LAYERS, pad_token_idx=PAD_TOKEN_IDX, device=device, dropout=0.0)

### Реализуйте обучение модели:
*  Не забудьте сдвинуть src и trg относительно друг друга.
*  Не забудьте про `clip_grad_norm_`
*  Данных очень много, для отладки лучше проходить только часть данных иначе этоха будет очень длинной

Получите `loss < 5.0` на тестовой выборке. 

Если модель обучается слишком быстро до значений <1.0 вы что-то напутали с данными.

In [11]:
model.train_loop(epochs=50, train_dataloader=train_dataloader, lr=LEARNING_RATE, save_path=SAVE_PATH, best_path=BEST_MODEL_PATH, patience=5)

Epoch 1/50


100%|██████████| 111/111 [00:01<00:00, 80.70it/s]


Train Loss: 5.39142, Eval Loss: 4.73945


100%|██████████| 111/111 [00:01<00:00, 80.81it/s]


Model saved to ./saved_models/model_epoch_1.pth at epoch 1
Best model saved to ./saved_models/best_model.pth
Epoch 2/50


...

Model saved to ./saved_models/model_epoch_49.pth at epoch 49
Best model saved to ./saved_models/best_model.pth
Epoch 50/50


100%|██████████| 111/111 [00:01<00:00, 84.20it/s]


Train Loss: 3.76616, Eval Loss: 3.95890


100%|██████████| 111/111 [00:01<00:00, 83.93it/s]


Model saved to ./saved_models/model_epoch_50.pth at epoch 50
Loss plot saved to ./saved_models/loss_plot_epoch_50.png
Model saved to ./saved_models/model_epoch_50.pth at epoch 50
Best model saved to ./saved_models/best_model.pth


# Генерация текста (5 баллов) 

Реализуйте функцию, которая продолжает текст.
1.   Переведите строчку в токены
2.   Реализуйте код который предсказывает вероятность следующей буквы
3.   Семплируйте следующую букву
4.   Повторяйте пункты 2-3 в цикле
5.   Преобразуйте токены в строчку

In [25]:
model.load_model(SAVE_PATH)

Loaded model from model_epoch_50.pth


In [13]:
model.continues_sentence("Я помню чудное мгновенье", dataset_maker.tokenizer)

'я помню чудное мгновенье, и я, как и в то время, как я был в состоянии, я не мог не забыть, но я не мог понять, что я'

In [14]:
model.continues_sentence("Мой дядя самых честных правил,", dataset_maker.tokenizer)

'мои дядя самых честных правил, а я, как и сам, не знаю, что я не могу... - я не знаю, что вы, - продолжал он, -'

In [23]:
model.continues_sentence("Четыре года потратил Деонардо на", dataset_maker.tokenizer)

'четыре года потратил деонардо на то, что он не мог не видеть, как он, как и в самом деле, и в то же время, когда он был в своем доме'

In [16]:

model.continues_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", dataset_maker.tokenizer)

'если сила плохих людеи в том, что они вместе, то хорошим людям, чтобы стать силои, надо, чтобы они не были в состоянии, чтобы не было ни одного слова, а не то, что они делают, и потому, что они не могут'